In [3]:
import datetime
import json
import re
from pymongo import MongoClient, TEXT, ASCENDING, DESCENDING
from pymongo.errors import BulkWriteError

# Hàm làm sạch giá
def clean_price(price_str):
    try:
        price_num = int(re.sub(r'[^\d]', '', price_str))
    except:
        price_num = 0
    return price_num

# Hàm xử lý dữ liệu sản phẩm
def process_product_data(raw_data):
    processed_data = []
    for item in raw_data:
        product = {}
        product['product_id'] = item.get('product_id', '').strip()
        product['name'] = item.get('name', '').strip()
        product['price'] = clean_price(item.get('price', '0'))
        product['description'] = item.get('description', '').strip()
        product['specs'] = item.get('specs', {})
        product['images'] = item.get('images', [])
        product['ratings'] = float(item.get('ratings', 0.0))
        product['reviews'] = int(item.get('reviews', 0))
        product['category'] = item.get('category', 'Uncategorized').strip()
        product['brand'] = item.get('brand', 'Unknown').strip()
        product['stock'] = int(item.get('stock', 0))
        # Định dạng ngày tháng
        created_at_str = item.get('created_at', '1970-01-01T00:00:00Z')
        updated_at_str = item.get('updated_at', '1970-01-01T00:00:00Z')
        try:
            product['created_at'] = datetime.datetime.fromisoformat(created_at_str.replace("Z", "+00:00"))
        except:
            product['created_at'] = datetime.datetime.utcnow()
        try:
            product['updated_at'] = datetime.datetime.fromisoformat(updated_at_str.replace("Z", "+00:00"))
        except:
            product['updated_at'] = datetime.datetime.utcnow()
        processed_data.append(product)
    return processed_data

# Hàm kết nối đến MongoDB
def connect_mongo(uri="mongodb://localhost:27017/", db_name="fptshop", collection_name="products"):
    client = MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]
    return collection

# Hàm chèn dữ liệu vào MongoDB với kiểm tra trùng lặp
def insert_products(collection, products):
    if not products:
        print("Không có sản phẩm nào để chèn.")
        return
    
    # Lấy danh sách product_id đã tồn tại trong MongoDB
    product_ids = [product['product_id'] for product in products]
    existing_ids = collection.find(
        {"product_id": {"$in": product_ids}}
    ).distinct("product_id")
    
    # Lọc ra các sản phẩm chưa tồn tại
    new_products = [product for product in products if product['product_id'] not in existing_ids]
    
    if new_products:
        try:
            result = collection.insert_many(new_products, ordered=False)
            print(f"Đã chèn thành công {len(result.inserted_ids)} sản phẩm mới.")
        except BulkWriteError as bwe:
            print("Một số sản phẩm không thể chèn do lỗi:", bwe.details)
    else:
        print("Tất cả các sản phẩm đã tồn tại trong cơ sở dữ liệu.")

# Hàm tạo chỉ mục
def create_indexes(collection):
    # Chỉ mục text cho tên sản phẩm
    collection.create_index([('name', TEXT)], name='name_text_index')
    
    # Chỉ mục cho category
    collection.create_index([('category', ASCENDING)], name='category_index')
    
    # Chỉ mục cho brand
    collection.create_index([('brand', ASCENDING)], name='brand_index')
    
    # Chỉ mục cho price và ratings
    collection.create_index([
        ('price', ASCENDING),
        ('ratings', DESCENDING)
    ], name='price_ratings_index')
    
    print("Đã tạo các chỉ mục thành công.")

# Hàm đọc dữ liệu từ file JSON
def read_data_from_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            raw_products = json.load(f)
        return raw_products
    except FileNotFoundError:
        print(f"File '{file_path}' không tồn tại.")
        return []
    except json.JSONDecodeError:
        print(f"File '{file_path}' không chứa dữ liệu JSON hợp lệ.")
        return []

# Hàm đọc dữ liệu từ file CSV (nếu cần)
import csv

def read_data_from_csv(file_path):
    products = []
    try:
        with open(file_path, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                # Cần xử lý các trường dữ liệu phù hợp
                product = {
                    "product_id": row.get('product_id', '').strip(),
                    "name": row.get('name', '').strip(),
                    "price": row.get('price', '0').strip(),
                    "description": row.get('description', '').strip(),
                    "specs": json.loads(row.get('specs', '{}')),
                    "images": json.loads(row.get('images', '[]')),
                    "ratings": float(row.get('ratings', 0.0)),
                    "reviews": int(row.get('reviews', 0)),
                    "category": row.get('category', 'Uncategorized').strip(),
                    "brand": row.get('brand', 'Unknown').strip(),
                    "stock": int(row.get('stock', 0)),
                    "created_at": row.get('created_at', '1970-01-01T00:00:00Z').strip(),
                    "updated_at": row.get('updated_at', '1970-01-01T00:00:00Z').strip()
                }
                products.append(product)
        return products
    except FileNotFoundError:
        print(f"File '{file_path}' không tồn tại.")
        return []
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return []


In [4]:
def main():
    # Kết nối MongoDB
    collection = connect_mongo()
    
    # Chèn sản phẩm đơn lẻ
    product = {
        "product_id": "12345",
        "name": "iPhone 14 Pro Max",
        "price": 29990000,
        "description": "iPhone 14 Pro Max với màn hình OLED 6.7-inch, chip A16 Bionic, RAM 6GB, và bộ nhớ trong 128GB. Hỗ trợ 5G, camera chất lượng cao, và pin 4000mAh.",
        "specs": {
            "Screen": "6.7-inch OLED",
            "Processor": "A16 Bionic",
            "RAM": "6GB",
            "Storage": "128GB",
            "Battery": "4000mAh",
            "Camera": "12MP + 12MP + 12MP"
        },
        "images": [
            "https://fptshop.com.vn/images/iphone14promax1.jpg",
            "https://fptshop.com.vn/images/iphone14promax2.jpg"
        ],
        "ratings": 4.8,
        "reviews": 1500,
        "category": "Smartphone",
        "brand": "Apple",
        "stock": 200,
        "created_at": datetime.datetime(2024, 1, 10, 10, 0, 0),
        "updated_at": datetime.datetime(2024, 12, 15, 15, 30, 0)
    }
    
    # Chèn một tài liệu
    result = collection.insert_one(product)
    print(f"Inserted product with _id: {result.inserted_id}")
    
    # Chèn nhiều tài liệu
    products = [
        {
            "product_id": "67890",
            "name": "Samsung Galaxy S23 Ultra",
            "price": 27990000,
            "description": "Samsung Galaxy S23 Ultra với màn hình Dynamic AMOLED 6.8-inch, chip Snapdragon 8 Gen 2, RAM 8GB, và bộ nhớ trong 256GB. Hỗ trợ 5G, camera đa chức năng, và pin 5000mAh.",
            "specs": {
                "Screen": "6.8-inch Dynamic AMOLED",
                "Processor": "Snapdragon 8 Gen 2",
                "RAM": "8GB",
                "Storage": "256GB",
                "Battery": "5000mAh",
                "Camera": "108MP + 12MP + 10MP + 10MP"
            },
            "images": [
                "https://fptshop.com.vn/images/galaxys23ultra1.jpg",
                "https://fptshop.com.vn/images/galaxys23ultra2.jpg"
            ],
            "ratings": 4.7,
            "reviews": 1200,
            "category": "Smartphone",
            "brand": "Samsung",
            "stock": 150,
            "created_at": datetime.datetime(2024, 2, 5, 11, 0, 0),
            "updated_at": datetime.datetime(2024, 12, 10, 14, 20, 0)
        },
        {
            "product_id": "54321",
            "name": "MacBook Pro 16-inch",
            "price": 79990000,
            "description": "MacBook Pro 16-inch với chip M2 Pro, RAM 16GB, và bộ nhớ trong 512GB SSD. Màn hình Retina 16-inch, Touch Bar, và pin lâu dài.",
            "specs": {
                "Screen": "16-inch Retina",
                "Processor": "M2 Pro",
                "RAM": "16GB",
                "Storage": "512GB SSD",
                "Battery": "10000mAh",
                "Graphics": "16-core GPU"
            },
            "images": [
                "https://fptshop.com.vn/images/macbookpro16_1.jpg",
                "https://fptshop.com.vn/images/macbookpro16_2.jpg"
            ],
            "ratings": 4.9,
            "reviews": 800,
            "category": "Laptop",
            "brand": "Apple",
            "stock": 50,
            "created_at": datetime.datetime(2024, 3, 15, 9, 30, 0),
            "updated_at": datetime.datetime(2024, 12, 12, 13, 45, 0)
        }
    ]
    
    insert_products(collection, products)
    
    # Chèn thêm dữ liệu từ file JSON
    additional_products = read_data_from_json('additional_products.json')
    processed_additional = process_product_data(additional_products)
    insert_products(collection, processed_additional)
    
    # Tạo chỉ mục
    create_indexes(collection)
    
    # Kiểm tra dữ liệu đã được chèn
    sample_product = collection.find_one({'product_id': '12345'})
    print("Sample Product:", sample_product)

if __name__ == "__main__":
    main()


Inserted product with _id: 67888c0da2db10df8a1faecd
Tất cả các sản phẩm đã tồn tại trong cơ sở dữ liệu.
Đã chèn thành công 2 sản phẩm mới.
Đã tạo các chỉ mục thành công.
Sample Product: {'_id': ObjectId('67888abba2db10df8a1faec9'), 'product_id': '12345', 'name': 'iPhone 14 Pro Max', 'price': 29990000, 'description': 'iPhone 14 Pro Max với màn hình OLED 6.7-inch, chip A16 Bionic, RAM 6GB, và bộ nhớ trong 128GB. Hỗ trợ 5G, camera chất lượng cao, và pin 4000mAh.', 'specs': {'Screen': '6.7-inch OLED', 'Processor': 'A16 Bionic', 'RAM': '6GB', 'Storage': '128GB', 'Battery': '4000mAh', 'Camera': '12MP + 12MP + 12MP'}, 'images': ['https://fptshop.com.vn/images/iphone14promax1.jpg', 'https://fptshop.com.vn/images/iphone14promax2.jpg'], 'ratings': 4.8, 'reviews': 1500, 'category': 'Smartphone', 'brand': 'Apple', 'stock': 200, 'created_at': datetime.datetime(2024, 1, 10, 10, 0), 'updated_at': datetime.datetime(2024, 12, 15, 15, 30)}
